In [ ]:
import joblib
import pandas as pd
import shap

df_test = pd.read_csv('test_storming_round.csv')


df_test['year_month'] = pd.to_datetime(df_test['year_month'], format="%m/%d/%Y")


df_test['year'] = df_test['year_month'].dt.year
df_test['month'] = df_test['year_month'].dt.month
drop_cols = ['month', 'policies_sold', 'next_month_policies', 'target']
feature_cols = [col for col in df_test.columns if col not in drop_cols]

X_test = df_test[feature_cols]


X_test = X_test.select_dtypes(include=['int', 'float', 'bool'])
X_test.drop(columns=['row_id'],inplace=True)
model = joblib.load('model.pkl')

probs = model.predict_proba(X_test)[:, 1]

threshold = 0.7
at_risk_agents = X_test[probs < threshold]
at_risk_probs = probs[probs < threshold]


explainer = shap.Explainer(model)
shap_values = explainer(at_risk_agents)



In [ ]:

at_risk_indices = X_test.index[probs < threshold]


agent_codes = df_test.loc[at_risk_indices, 'agent_code'].reset_index(drop=True)


at_risk_agents = at_risk_agents.reset_index(drop=True)
at_risk_agents['agent_code'] = agent_codes




In [ ]:
at_risk_agents = at_risk_agents.copy()
at_risk_agents['risk_score'] = at_risk_probs


at_risk_agents['top_reason_1'] = ""
at_risk_agents['top_reason_2'] = ""
at_risk_agents['top_reason_3'] = ""


for i in range(len(at_risk_agents)):
    print(f"\nAgent {at_risk_agents['agent_code'].iloc[i]} (Risk Score: {at_risk_agents['risk_score'].iloc[i]:.2f}):")
    
   
    shap_vals = shap_values[i].values
    feature_names = at_risk_agents.columns.drop(['risk_score', 'agent_code', 'top_reason_1', 'top_reason_2', 'top_reason_3'])
    
   
    top_indices = abs(shap_vals).argsort()[::-1][:3]
    
    for j, idx in enumerate(top_indices):
        feature = feature_names[idx]
        impact = shap_vals[idx]
        direction = "↑" if impact > 0 else "↓"
        print(f"  - {feature}: {direction}{abs(impact):.4f}")
        
        # Assign top reasons
        at_risk_agents.at[i, f'top_reason_{j+1}'] = feature
    
    at_risk_agents.to_csv('at_risk_reasons.csv',index=False)



Agent 27f35403 (Risk Score: 0.65):
  - unique_proposals_last_21_days: ↓0.5047
  - unique_proposals_last_15_days: ↓0.2713
  - agent_age: ↓0.1942

Agent f2fcfe25 (Risk Score: 0.68):
  - agent_age: ↓0.4285
  - net_income: ↓0.3615
  - unique_quotations: ↓0.3215

Agent 6452bd08 (Risk Score: 0.68):
  - net_income: ↓0.3143
  - unique_proposals_last_21_days: ↓0.2142
  - number_of_policy_holders: ↓0.2118

Agent 60b8427e (Risk Score: 0.63):
  - ANBP_value: ↓0.4097
  - net_income: ↓0.2575
  - number_of_policy_holders: ↓0.2227

Agent 6c9d3c40 (Risk Score: 0.64):
  - ANBP_value: ↓0.3403
  - net_income: ↓0.1916
  - unique_proposals_last_15_days: ↓0.1641

Agent d360368b (Risk Score: 0.59):
  - unique_customers_last_15_days: ↓0.3286
  - new_policy_count: ↓0.3272
  - unique_quotations: ↓0.2486

Agent f7136abb (Risk Score: 0.52):
  - number_of_cash_payment_policies: ↓0.5020
  - net_income: ↓0.3602
  - unique_proposals_last_21_days: ↓0.2976

Agent 5724f683 (Risk Score: 0.56):
  - unique_proposals_last_2

In [ ]:
import pandas as pd


feature_plans = {
    "net_income": "Provide personalized coaching to help the agent identify and pursue higher-value opportunities. Review past sales to identify missed income potential.",
    "ANBP_value": "Coach the agent on improving business quality and premium value by focusing on client needs and cross-selling opportunities.",
    "unique_proposals_last_21_days": "Work with the agent on optimizing proposal timing and targeting, ensuring consistency in lead follow-up and conversion strategy.",
    "agent_age": "Offer age-specific mentoring—young agents may benefit from product knowledge training, while older ones might need digital tool support.",
    "number_of_cash_payment_policies": "Educate the agent on the risks and best practices of cash handling, and promote digital payment solutions for better client tracking.",
    "number_of_policy_holders": "Support the agent in building long-term client relationships and retention strategies to grow a loyal customer base.",
    "unique_proposals_last_15_days": "Guide the agent to review proposal success rates and fine-tune their approach to ensure productive outreach.",
    "unique_quotations": "Mentor on quote effectiveness—how to target the right clients and ensure quotations are tailored and likely to convert.",
    "unique_customers_last_15_days": "Encourage the agent to increase and diversify outreach, and provide tools for efficient prospecting in new segments.",
    "unique_proposal": "Review historical sales cycles with the agent to reignite consistent proposal generation and uncover potential bottlenecks.",
    "unique_customers": "Mentor the agent on portfolio diversification and expanding customer reach, using referral strategies and community engagement.",
    "unique_customers_last_21_days": "Re-engage agent in consistent customer interactions and offer time-management coaching to maintain relationship momentum.",
    "new_policy_count": "Provide coaching on sustainable business practices and verifying client intent to avoid short-term gain but long-term loss.",
    "unique_quotations_last_7_days": "Help the agent reflect on recent quotation patterns and adjust for better targeting and time efficiency.",
    "unique_proposals_last_7_days": "Offer real-time coaching to improve proposal quality and address any rushed or repetitive approaches.",
    "unique_quotations_last_21_days": "Collaborate with the agent on streamlining the quoting process, ensuring follow-through and effective time use.",
    "unique_quotations_last_15_days": "Provide coaching sessions focused on improving recent quotation outcomes and understanding client objections.",
    "unique_customers_last_7_days": "Help the agent build consistent weekly routines to ensure they stay connected with a healthy mix of prospects and clients."
}


df = pd.read_csv('at_risk_reasons.csv') 


def generate_coaching_plan(row):
    coaching_items = []
    for feature in [row["top_reason_1"], row["top_reason_2"], row["top_reason_3"]]:
        plan = feature_plans.get(feature, f"[No mentoring plan defined for {feature}]")
        coaching_items.append(f"- {plan}")
    return "\n".join(coaching_items)


df["coaching_plan"] = df.apply(generate_coaching_plan, axis=1)


df[["agent_code", "coaching_plan"]].to_csv("agent_coaching_plans.csv", index=False)




✅ Coaching plans saved to 'agent_coaching_plans.csv'
